In [2]:
from bertopic import BERTopic
import json
import pandas as pd
from  services.influencers_service import  InfluencersService
from shared.mongo import MongoConnection
from gensim.parsing.preprocessing import STOPWORDS

In [24]:
mongo_connection=MongoConnection()
influencers_service = InfluencersService(mongo_connection)
bios=[]
posts=influencers_service.get_posts()
influencers=influencers_service.get_influencers()
custom_stopwords={'null','like','com','www','gmail','hotmail'}
for influencer in influencers:
    info_string=""
    if influencer['Bio']!= 'NULL':
        info_string+=influencer['Bio']
    if influencer['Category']!= 'NULL':
        info_string+=influencer['Category']
    if info_string!= "":
        for stopword in custom_stopwords:
            info_string=info_string.replace(stopword,"")
        bios.append(info_string)



2024-05-29 01:14:21,895 - INFO - MongoDB connection established successfully
2024-05-29 01:14:22,909 - INFO - Fetched 20316 posts
2024-05-29 01:14:23,017 - INFO - Fetched 38112 influencers


In [25]:
topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2")


In [5]:
# from bertopic.representation import KeyBERTInspired
# 
# # Fine-tune your topic representations
# representation_model = KeyBERTInspired()
# topic_model = BERTopic(representation_model=representation_model)

In [26]:
topics, probs = topic_model.fit_transform(bios)
print(topics)
print(probs)

2024-05-29 01:14:31,634 - INFO - Use pytorch device_name: cpu
2024-05-29 01:14:31,635 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


[-1, 8, -1, 31, 0, -1, 4, 24, 43, -1, 30, 30, 30, -1, 30, -1, 43, 87, 30, 43, -1, 30, 87, 109, -1, 157, -1, -1, -1, -1, 1, 42, -1, -1, -1, 28, -1, 19, 48, 0, 16, -1, 68, 31, -1, -1, 142, 52, -1, -1, 55, 91, -1, -1, -1, -1, -1, -1, 0, 15, -1, -1, -1, -1, -1, -1, 7, 15, 17, 1, -1, 10, 33, -1, -1, -1, 0, -1, 75, 111, 174, 15, -1, 44, 143, 85, -1, -1, -1, 1, -1, -1, -1, 91, 8, -1, 114, 25, 200, 41, 78, 7, -1, -1, -1, 4, 0, -1, 0, -1, 28, 47, -1, 0, -1, 0, 0, -1, 22, -1, -1, -1, 44, -1, -1, -1, 2, 157, -1, 11, 102, 99, 2, 15, 16, -1, -1, -1, 43, 30, 30, 30, -1, 49, 30, 44, -1, 83, 30, -1, 30, 30, 43, 30, 30, -1, 7, 30, 30, 30, 30, 30, 43, 30, -1, 24, -1, -1, -1, -1, -1, -1, -1, -1, -1, 23, -1, 12, -1, -1, 57, 1, 32, 12, 123, -1, 39, -1, 92, 0, -1, -1, 18, 77, 10, 42, 26, 76, -1, -1, 54, 33, -1, -1, 22, -1, 28, 33, 21, 30, 15, -1, 13, -1, 15, 15, -1, 21, -1, -1, -1, 39, 92, 3, 1, -1, 36, -1, 26, 27, 53, -1, 1, 31, -1, 6, -1, 64, 55, -1, -1, 39, 1, -1, 4, 39, -1, 144, 53, -1, -1, 53, -1, -1, 

In [7]:
topic_model.get_topic_info()
topic_model.get_topic(topic=0)
topic_model.get_representative_docs(-1)


['official Instagram account # DisneyStyle Fashion , diy , news , beauty Disney fan . Explore Instagram feed :Personal Goods & General Merchandise Stores',
 'well - be workshop 18.10.18 content creator food photography healthy , quick & simple recipe sport & health coach online - shopCreators & Celebrities',
 'travel & lifestyle blog event specialist currently london travel , interior , beauty & fashion']

In [30]:
# df = pd.DataFrame({'topic': topics, 'document': bios})
# topic_model.reduce_topics(bios, nr_topics=230)
topic_model.merge_topics(bios,[3,6,7,4])

In [31]:
topic_model.visualize_topics()

In [32]:
topic_model.visualize_barchart()


In [36]:
topic_model.save("/model/bert_4", serialization="safetensors")
BERTopic.load("/model/bert_4")


SafetensorError: Error while serializing: IoError(Os { code: 1224, kind: Uncategorized, message: "L’opération demandée n’a pu s’accomplir sur un fichier ayant une section mappée utilisateur ouverte." })

In [38]:
new_topics, new_prob=topic_model.transform(documents="sport")
print(new_topics,new_prob)

[-1] [0.]
